TODO: 1. For a given test set (word,sentence,document) get the ranking of concepts
      2. Find similarity between two words, sentence and document (using cosine product)

In [1]:
from gensim import *

import json 
data = [json.loads(_) for _ in open('wiki-100k-sample.json')]
from gensim.parsing import preprocess_string
corpus_data = []
for article in data[:100]:
    document = preprocess_string(str(article['section_texts']))
    corpus_data.append(document)
    


In [2]:
## inbuilt tfidf
dictionary = corpora.Dictionary(corpus_data)
dictionary.save('/tmp/corpus_data.dict') 

raw_corpus = [dictionary.doc2bow(t) for t in corpus_data]

corpora.MmCorpus.serialize('/tmp/corpus_data.mm', raw_corpus) # store to disk

dictionary = corpora.Dictionary.load('/tmp/corpus_data.dict')

corpus = corpora.MmCorpus('/tmp/corpus_data.mm')



In [3]:
import numpy as np

In [4]:
#Matrix initialization
b = np.zeros([corpus.num_docs,corpus.num_terms])

In [5]:
from gensim import sklearn_api
sk_tfidf = sklearn_api.tfidf.TfIdfTransformer()

In [6]:
a = sk_tfidf.fit_transform(corpus)
# a

In [7]:
for i in range(len(a)):
    for j in a[i]:
        b[i,j[0]] = j[1]
matrix_T = b

In [8]:
matrix_T

array([[ 0.00955392,  0.04469996,  0.0050507 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.00178742,  0.00141374, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.00315909,  0.        ,  0.00334012, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.01364905,
         0.01364905,  0.01364905]])

In [9]:
import pandas as pd
esa = pd.DataFrame(matrix_T,index=[i for i in range(np.shape(matrix_T)[0])],columns=[dictionary.get(i) for i in range(np.shape(matrix_T)[1])])
esa = esa.transpose()
esa.head()


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
shop,0.009554,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.008132,0.000000,0.000000,0.0,0.003159,0.0,0.0
code,0.044700,0.0,0.001787,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.006515,0.003425,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
consider,0.005051,0.0,0.001414,0.0,0.0,0.0,0.0,0.0,0.0,0.008764,...,0.000000,0.002577,0.000000,0.002150,0.004548,0.004181,0.0,0.003340,0.0,0.0
consum,0.018272,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.002304,0.000000,0.001633,0.007777,0.000000,0.007563,0.0,0.000000,0.0,0.0
interchang,0.011163,0.0,0.003125,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0


In [10]:
test_1 = "shop"
test_2 = "code"
test_3 = "consider consum interchang"

test =[test_1,test_2,test_3]

def get_word_rank(string):
    words = string.split()
    vector = esa.loc[words[0]].as_matrix()
    for i in range(1,len(words)):
        vector = vector + esa.loc[words[i]].as_matrix()
    return vector/len(words)

def get_similarity(t1,t2):
    doc1 = get_word_rank(t1)
    doc2 = get_word_rank(t2)
    sim = np.dot(doc1,doc2)/np.sqrt((np.dot(doc1,doc1)*np.dot(doc2,doc2)))
    return sim


for i in test:
    for j in test:
        print "doc1:",i
        print "doc2:", j
        print get_similarity(i,j)
        print
    


doc1: shop
doc2: shop
1.0

doc1: shop
doc2: code
0.247207511596

doc1: shop
doc2: consider consum interchang
0.447798096915

doc1: code
doc2: shop
0.247207511596

doc1: code
doc2: code
1.0

doc1: code
doc2: consider consum interchang
0.34554380735

doc1: consider consum interchang
doc2: shop
0.447798096915

doc1: consider consum interchang
doc2: code
0.34554380735

doc1: consider consum interchang
doc2: consider consum interchang
1.0



## manual tfidf 

In [11]:

mydoclist = corpus_data
from collections import Counter

for doc in mydoclist[:10]:
    tf = Counter()
    for word in doc:
        tf[word] +=1
#     print tf.items()

In [12]:
import string #allows for format()
mydoclist = corpus_data   
def build_lexicon(corpus):
    lexicon = set()
    for doc in corpus:
        lexicon.update([word for word in doc])
    return lexicon

def tf(term, document):
  return freq(term, document)

def freq(term, document):
  return document.count(term)

vocabulary = build_lexicon(mydoclist[:10])

doc_term_matrix = []
# print 'Our vocabulary vector is [' + ', '.join(list(vocabulary)) + ']'
for doc in mydoclist[:10]:

    tf_vector = [tf(word, doc) for word in vocabulary]
    tf_vector_string = ', '.join(format(freq, 'd') for freq in tf_vector)
#     print 'The tf vector for Document %d is [%s]' % ((mydoclist.index(doc)+1), tf_vector_string)
    doc_term_matrix.append(tf_vector)
    
    
# print 'All combined, here is our master document term matrix: '
# print doc_term_matrix

In [13]:
import scipy

In [14]:
sparse =scipy.sparse.coo_matrix(doc_term_matrix)

In [15]:
sparse.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 5, ..., 2, 3, 1],
       ..., 
       [0, 2, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [16]:
# esa = pd.DataFrame(sparse.toarray(),index=[i for i in range(np.shape(matrix_T)[0])],columns=[dictionary.get(i) for i in range(np.shape(matrix_T)[1])])
esa = pd.DataFrame(sparse.toarray())

In [17]:
esa

,0,1,2,3,4,5,6,7,8,9,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,13,0,0,0
2,1,0,5,1,0,1,0,2,1,0,...,0,1,2,0,0,0,1,2,3,1
3,0,0,0,0,0,0,1,0,0,0,...,1,0,0,2,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,2,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
9,0,0,0,0,0,0,0,0,1,3,...,0,0,2,0,0,0,0,0,0,0


In [18]:
def numDocsContaining(word, doclist):
    doccount = 0
    for doc in doclist:
        if freq(word, doc) > 0:
            doccount +=1
    return doccount 

def idf(word, doclist):
    n_samples = len(doclist)
    df = numDocsContaining(word, doclist)
    return np.log(n_samples / 1+df)

my_idf_vector = [idf(word, mydoclist[:10]) for word in vocabulary]

In [19]:
def build_idf_matrix(idf_vector):
    idf_mat = np.zeros((len(idf_vector), len(idf_vector)))
    np.fill_diagonal(idf_mat, idf_vector)
    return idf_mat

my_idf_matrix = build_idf_matrix(my_idf_vector)

In [20]:
my_idf_matrix

array([[ 2.39789527,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  2.39789527,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  2.39789527, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  2.39789527,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         2.39789527,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  2.39789527]])

In [21]:


def l2_normalizer(vec):
    denom = np.sum([el**2 for el in vec])
    return [(el / np.sqrt(denom)) for el in vec]


In [25]:
doc_term_matrix_tfidf = []

#performing tf-idf matrix multiplication
for tf_vector in doc_term_matrix:
    doc_term_matrix_tfidf.append(np.dot(tf_vector, my_idf_matrix))

#normalizing
doc_term_matrix_tfidf_l2 = []
for tf_vector in doc_term_matrix_tfidf:
    doc_term_matrix_tfidf_l2.append(l2_normalizer(tf_vector))
    
# doc_term_matrix_tfidf_l2

## second order implementation

In [22]:
### sample links data 

links = { 1: [2,3,4,5],2:[3,5],3:[],4:[2],5:[1]}

In [23]:
esa2 = esa
alpha = 0.5
for i in links.keys():
    for j in links[i]:
        esa2[i] = esa[i] + alpha*esa[j]

In [24]:
esa2

,0,1,2,3,4,5,6,7,8,9,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922
0,0,0.5,0.5,0,0.25,1.25,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0.0,0.0,0,0.00,0.00,0,0,1,0,...,0,0,0,0,1,1,13,0,0,0
2,1,3.5,6.0,1,3.00,2.75,0,2,1,0,...,0,1,2,0,0,0,1,2,3,1
3,0,0.0,0.0,0,0.00,0.00,1,0,0,0,...,1,0,0,2,0,0,0,0,0,0
4,0,0.0,0.0,0,0.00,0.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0.0,0.0,0,0.00,0.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0.0,0.0,0,0.00,0.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,3.0,0.0,0,2.00,1.50,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0.0,0.0,0,0.00,0.00,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
9,0,0.0,0.0,0,0.00,0.00,0,0,1,3,...,0,0,2,0,0,0,0,0,0,0
